In [1]:
import json
import pandas as pd
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora, models
from gensim.models import TfidfModel

/home/g/grad/lpang/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/home/g/grad/lpang/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
data = pd.read_csv("covid_data.csv", header=None)

In [3]:
data.head(10)

,0,1,2,3,4,5,6
0,paper_id,title,authors,abstract,body,publish_time,text
1,5e0c586f047ff909c8ed3fe171c8975a90608d08,Neutralizing antibodies against porcine epidem...,"Gustafson, Kyle B.; Proctor, Joshua L.",Background: Porcine epidemic diarrhea virus (P...,"Porcine epidemic diarrhea virus (PEDV), which ...",2018 Aug 30,"['neutralize', 'antibody', 'porcine', 'epidemi..."
2,1579fbff7af9b156c6f49fee0526e48f852ea460,A Recombinant Newcastle Disease Virus (NDV) Ex...,"Gustafson, Kyle B.; Proctor, Joshua L.","Currently, live-attenuated IBV vaccines are us...","Generation of rNDVs expressing S1, S2 or S pro...",2018 Aug 10,"['recombinant', 'newcastle', 'disease', 'virus..."
3,e0668c4b793d0cad26639b070819334a94648123,GP-confirmed complete Achilles tendon rupture ...,"S.J,Davis;A,Lott;E,Besada",NaN,The incidence of complete Achilles tendon rupt...,NaN,"['gp-confirmed', 'complete', 'achilles', 'tend..."
4,38aa050ad79d8a1d7022c33535255ce9d47914e5,Potent Inhibition of Junín Virus Infection by ...,"Gustafson, Kyle B.; Proctor, Joshua L.",The new world arenavirus Junín virus (JUNV) is...,Arenaviruses are enveloped RNA viruses with bi...,2014 Jun 05,"['potent', 'inhibition', 'junín', 'virus', 'in..."
5,61722c462b054f36461375e96e502cbf22648c04,2 convergent Research center for emerging Viru...,"Gustafson, Kyle B.; Proctor, Joshua L.",and subtropical countries and is a significant...,"In this study, the anti-dengue activity of nic...",2019 Jun 18,"['2', 'convergent', 'research', 'center', 'eme..."
6,7107f088cbed45d8a06a026276ccf4d602d50f10,Microglia Play a Major Role in Direct Viral-In...,"Gustafson, Kyle B.; Proctor, Joshua L.",Microglia are the resident macrophage-like pop...,Microglia are specialized macrophages of the C...,2013 Jun 20,"['microglia', 'play', 'major', 'role', 'direct..."
7,6110a4ffed7e5b3cd92d1e7e40a80689f91edc98,Desafios para o enfrentamento da pandemia COVI...,"E.A,Medeiros;),;Eduardo,Alexandrino;Servolo,Me...",W e are living the most important pandemic in ...,"calendars, including schools of Medicine, with...",NaN,"['desafios', 'para', 'o', 'enfrentamento', 'da..."
8,2188208efc7ac8e7721dfc5cbea787d5b62c3537,Alternative conformations of a major antigenic...,"Gustafson, Kyle B.; Proctor, Joshua L.",The respiratory syncytial virus (RSV) fusion (...,a1111111111 a1111111111 a1111111111 a111111111...,2019 Jul 15,"['alternative', 'conformation', 'major', 'anti..."
9,dd74c8f2961dc716ec9d0c412206c88e0cb9b314,Information theory-based algorithm for in sili...,"Gustafson, Kyle B.; Proctor, Joshua L.",A new algorithm for assessing similarity betwe...,PCR technique is widely used in the molecular ...,2005 Jul 26,"['information', 'theory-based', 'algorithm', '..."


In [4]:
pd.DataFrame(data[6])

,6
0,text
1,"['neutralize', 'antibody', 'porcine', 'epidemi..."
2,"['recombinant', 'newcastle', 'disease', 'virus..."
3,"['gp-confirmed', 'complete', 'achilles', 'tend..."
4,"['potent', 'inhibition', 'junín', 'virus', 'in..."
...,...
9914,"['lipidome', 'profile', 'postnatal', 'day', '2..."
9915,"['secondary', 'syphilis', 'cali', ',', 'colomb..."
9916,"['replication', 'shed', 'mers-', 'cov', 'jamai..."
9917,"['role', 'erk1/2', 'signal', 'pathway', 'repli..."


In [6]:
pd.DataFrame(data[6]).values.reshape((9919,))

array(['text',
       '[\'neutralize\', \'antibody\', \'porcine\', \'epidemic\', \'diarrhea\', \'virus\', \'block\', \'virus\', \'attachment\', \'internalization\', \'porcine\', \'epidemic\', \'diarrhea\', \'virus\', \'(\', \'pedv\', \')\', \',\', \'belong\', \'alphacoronavirus\', \'genus\', \'coronaviridae\', \'family\', \',\', \'etiological\', \'agent\', \'porcine\', \'epidemic\', \'diarrhea\', \'(\', \'ped\', \')\', \'cause\', \'enteric\', \'disease\', \'affect\', \'age\', \'swine\', \'[\', \'1\', \',\', \'2\', \']\', \'.\', \'clinical\', \'presentation\', \'complication\', \'infection\', \'characterize\', \'acute\', \'vomiting\', \',\', \'dehydration\', \',\', \'watery\', \'diarrhea\', \',\', \'high\', \'mortality\', \'suck\', \'piglet\', \'[\', \'3\', \']\', \'indistinguishable\', \'infection\', \'either\', \'transmissible\', \'gastroenteritis\', \'virus\', \'(\', \'tgev\', \')\', \'porcine\', \'enteric\', \'alphacoronavirus\', \'(\', \'peav\', \')\', \'[\', \'4\', \',\', \'5\', \

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [9]:
n_features = 2000

tf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=["a", "an", "and", "are", "as", "at", "be", "but", "by",
                                              "for", "if", "in", "into", "is", "it",
                                              "no", "not", "of", "on", "or", "such",
                                              "that", "the", "their", "then", "there", "these",
                                              "they", "this", "to", "was", "will", "with"],
                                max_df = 0.99,
                                min_df = 0.002) #remove rare terms
print("vectorization is complete")
tf = tf_vectorizer.fit_transform(pd.DataFrame(data[6]).values.reshape((9919,)))

print(tf.shape)
print(tf)

vectorization is complete
(9919, 2000)
  (0, 1825)	1.0
  (1, 78)	0.00393920289571578
  (1, 1287)	0.0033044896743536665
  (1, 471)	0.004524519688588147
  (1, 1648)	0.00567131316791433
  (1, 726)	0.005028796007502644
  (1, 1106)	0.007301709058733906
  (1, 1056)	0.004188576750748068
  (1, 1458)	0.0071640515141933305
  (1, 365)	0.007398486650490462
  (1, 487)	0.006975372072072418
  (1, 1563)	0.006302491113702079
  (1, 651)	0.020336627333016635
  (1, 1910)	0.017133599699129483
  (1, 1)	0.006658645139239899
  (1, 3)	0.009274111135171932
  (1, 1924)	0.0072796560240417025
  (1, 1686)	0.0032809454493527134
  (1, 584)	0.007083035205655211
  (1, 1428)	0.0030378251553544134
  (1, 437)	0.004315700856289748
  (1, 983)	0.00606336713169817
  (1, 1565)	0.002866871714744819
  (1, 1003)	0.006078121976519722
  (1, 398)	0.005656092895165877
  :	:
  (9918, 561)	0.009557165816871806
  (9918, 1179)	0.04044789413336037
  (9918, 1593)	0.016198271364817187
  (9918, 1750)	0.02418306285349472
  (9918, 629)	0.01251

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
#set number of topics
n_topics = 3

lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=100,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)
lda.fit(tf)

#show topics model.topic_word_
print(lda.components_)
print(lda.components_.shape)                         

#calculate complexity
print(u'complexity：')
print(lda.perplexity(tf,sub_sampling = False))  

#set top words
def print_top_words(model, tf_feature_names, n_top_words):
    for topic_idx,topic in enumerate(model.components_):    # lda.component相当于model.topic_word_
        print('Topic #%d:' % topic_idx)
        print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))

#set number of top words
n_top_words = 20                                       
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


#-------------------  visulization  --------------------- 
import pyLDAvis
import pyLDAvis.sklearn

#pyLDAvis.enable_notebook()

data = pyLDAvis.sklearn.prepare(lda,tf,tf_vectorizer)
print(data)

#show data
# pyLDAvis.show(data)

pyLDAvis.save_html(data, 'covid_full_data_new.html')

[[33.42317676 18.84176998 35.07992497 ... 68.300612   95.34891448
  66.03301646]
 [ 0.38217064  0.38616965  0.3832588  ...  0.36871879  0.37757838
   0.38713536]
 [32.16608395 22.43459174  9.94701052 ...  0.37148529  2.46202516
   0.40416008]]
(3, 2000)
困惑度：
1799.6261377160874
Topic #0:
cell protein virus use sequence gene viral fig infection rna mouse expression figure show antibody 10 et al strain ml
Topic #1:
zikv denv dengue chikv mosquito wnv zika microglia flavivirus ns1 jev encephalitis 2016 cns african west woman brain fetal neuron
Topic #2:
patient study case health disease use infection influenza virus respiratory data outbreak child model risk age year hospital covid population


/home/g/grad/lpang/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.025889 -0.110923       1        1  57.113042
2     -0.130111  0.037079       2        1  39.633676
1      0.104222  0.073844       3        1   3.253282, topic_info=     Category        Freq          Term       Total  loglift  logprob
1994  Default  223.000000          zikv  223.000000  30.0000  30.0000
555   Default  187.000000          denv  187.000000  29.0000  29.0000
553   Default  135.000000        dengue  135.000000  28.0000  28.0000
364   Default  803.000000          cell  803.000000  27.0000  27.0000
382   Default  107.000000         chikv  107.000000  26.0000  26.0000
...       ...         ...           ...         ...      ...      ...
1387   Topic3    1.645427        plaque   39.459562   0.2482  -7.6464
443    Topic3    1.469365  complication   27.361155   0.5012  -7.7596
765    Topic3    1.852522         fever   69.271615  -0.1